# Cvičenie 10: Autoenkódery

Na dnešnom cvičení budeme sa zaoberať so špeciálnym prípadom nekontrolovaného učenia, ktorým sú autoenkódery. Autoenkódery sú neurónové siete, ktoré slúžia najmä na kompresiu údajov, avšak oproti tradičným metódam kompresie (RAR, ZIP, atď.) ich nepotrebujete vopred naprogramovať. Ich ďalšou vlastnosťou je, že sú špecificky určené na kompresiu údajov podobných dátam, na ktorých bola sieť natrénovaná. Tým pádom kompresia nie je univerzálne použiteľná, ale na druhej strane je často efektívnejšia, aj keď, podobne ako pri všetkých metódach kompresie, aj pri autoenkóderoch dochádza k strate pri kompresii.

Z topologického hľadiska sú autoenkódery zvyčajne symetrické (síce to nie je nevyhnutné, ale v praxi takéto siete sa trénujú jednoduchšie). Cieľom autoenkóderov je kopírovať vstup na výstupe tak, že skryté vrstvy majú menej neurónov ako vstup, resp. výstup. Autoenkódery teda majú dve časti: prvá je enkóder, ktorý skomprimuje údaje, kým druhá časť, teda dekóder ich reprodukuje zo zakódovaného tvaru.

![](figures/lab10/10.1-autoencoder.png)

Na dnešnom cvičení implementujeme jeden autoenkóder v Kerase a vyskúšame jeho funkčnosť. Toto nám umožní ukázať, ako sa definujú siete v Kerase ktoré potom rozdelíte na dve (alebo viac) časti. Kostru riešenia nájdete [tu](codes/lab10.zip).

## 1. Načítanie potrebných knižníc

V riešení využijeme už známe knižnice `keras`, `numpy` a `matplotlib`. Ako dataset použijeme MNIST dataset, ktorý obsahuje obrázky rukou písaných číslic. Tento dataset je ďalší štandardný dataset (najmä pre konvolučné siete), takže ho nájdeme v knižnici keras. Pri prvom spustení kódu sa vám stiahne dataset, takže prvé spustenie môže trvať dlhšie.

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

## 2. Vizualizácia výsledkov

V kóde máte pripravenú metódu `plot_results`, ktorá pomocou `matplotlib` vizualizuje výstup z jednotlivých častí siete (vstup, enkódovaný obraz, dekódovaný obraz). Metóda ukáže niekoľko (*n*) ukážkových vstupov, ich zakódovaný tvar, a rekonštruovaný obraz.

In [ ]:
def plot_results(x_test, encoded_imgs, decoded_imgs, n=10):
    plt.figure(figsize=(40, 4))
    for i in range(n):
        # display original
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display encoded
        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(encoded_imgs[i].reshape(8, 4))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(3, n, i + 1 + n * 2)
        plt.imshow(decoded_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

## 3. Predspracovanie údajov

Keďže dnes nebudeme používať konvolúciu, dataset musíme upraviť tak, aby obrázky boli reprezentované ako jednorozmerné vektory. Originálne MNIST dataset obsahuje obrázky *28x28*, ktoré mi prekonvertujeme na tvar *(1x)784*. Pred tým ale pixely normalizujeme, t.j. pretypujeme ich na numpyovský float - hodnoty od 0 po 255 namapujeme do intervalu 0 až 1. Toto nám zabezpečí rýchlejšiu konvergenciu siete.

Všimnite si, že z datasetu nenačítame labely (nahradíme ich znakom `_`). Dôvodom je, že autoencodery by mali mať na výstupe rovnakú hodnotu ako na vstupe, teda stačia nám vstupné údaje z datasetu - preto sa považujú za príklad nekontrolovaného učenia, aj keď ich v princípe trénujeme akokeby sme robili kontrolované učenie

In [ ]:
(mnist_train, _), (mnist_test, _) = mnist.load_data()

x_train =  # TODO: normalize data to interval 0-1
x_test =  # TODO: normalize data to interval 0-1
x_train =  # TODO: reshape to flattened input: (len, 28, 28) -> (len, 784)
x_test =  # TODO: reshape to flattened input: (len, 28, 28) -> (len, 784)

## 4. Definícia autoenkódera

V ďalšom kroku zadefinujeme autoenkóder s jednou skrytou vrstvou (neskôr tento kód môžete opraviť a rozšíriť model o ďalšie vrstvy).

### 4.1. Definícia vrstiev

Keďže niektoré vrstvy chceme využiť niekoľkokrát a vo viacerých modeloch, namiesto toho, aby sme najprv vytvorili model a postupne sme doňho pridávali vrstvy, najprv zadefinujeme vrstvy, ktoré spojíme pomocou parametrov v tvare:

```dalsia_vrstva = Typ_vrstvy(parametre)(predosla_vrstva)```

In [ ]:
input_img =  # TODO: define input layer with size

encoded =  # TODO: define hidden layer - 32 neurons, relu; connect to input
decoded =  # TODO: define output layer, same as input, use sigmoid act

### 4.2. Definícia modelov

Ak už máme vrstvy zadefinované, môžeme pristúpiť k definícii modelov. V tomto kroku zadefinujeme tri modely: celý autoenkóder, enkóder, a dekóder. Modely autoenkódera a enkódera vieme zadefinovať na základe ich vstupu a výstupu v tvare:

```model_object = Model(vstupna_vrstva, vystupna_vrstva)```

In [ ]:
autoencoder =  # TODO: define entire model
encoder =  # TODO: define encoder part

Definícia modelu dekódera je menej priamočiara. Jeho vstupom je síce výstup enkódera, ale nevieme priamo použiť vrstvy z autoencodera, keďže tie sú navzájom prepojené. Namiesto toho vytvoríme novú vstupnú vrstvu a následne pridáme ďalšie potrebné vrstvy (v tomto prípade iba poslednú vrstvu autoenkódera).

In [ ]:
encoded_input =  # TODO: define new input layer for encoded code
decoder_layer =  # TODO: load last autoencoder layer, connect with previous
decoder =  # TODO: define decoder model

### 4.3. Trénovanie autoenkódera

Neostáva nám nič iné, len natrénovať našu sieť. Pridajte chýbajúce parametre do volaní funkcií nasledovne:
* optimizer - napr. adadelta
* loss - napr. binary_crossentropy
* input - vstup autoencodera
* output - očakávaný výstup autoencodera
* epochs - napr. 25
* batch_size - napr. 256
* shuffle - nastavte na `True` pre náhodné poradie v trénovacej množine
* verbose - nastavte na 2 pre jednoduchší výpis progresu

In [ ]:
autoencoder.compile(  # TODO: add parameters
)
autoencoder.fit(  # TODO: add parameters
)

### 4.4. Testovanie autoenkódera

Ak už máme autoencoder natrénovaný, môžeme vizualizovať jeho funkčnosť. Zavoláme funkciu `predict` nad testovacou množinou. Keďže chceme vizualizovať vstup, zakódovanú reprezentáciu, aj výstup autoenkódera, funkciu potrebujeme zavolať aj pre enkóder aj pre samotný autoenkóder. Ak chcete testovať zvlášť aj dekóder, upravte kód podľa potreby.

In [ ]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

plot_results(x_test, encoded_imgs, decoded_imgs)

Zobrazia sa nám tri riadky obrázkov, kde prvý riadok je vstup, druhý je zakódovaný tvar, a posledný riadok obsahuje rekonštruované obrázky, t.j. výstup autoenkódera, napr. pri nastaveniach z bodu 4.3:

![](figures/lab10/10.2-shallow-results.jpg)

Ukážkové riešenie nájdete [na tomto odkaze](solutions/lab10-autoencoder-solution.py).